In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Nov  2 02:00:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    22W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pip install torch

In [ ]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-mm_4yqbd
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-mm_4yqbd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
! pip install datasets  rouge-score nltk

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
! sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
import transformers

print(transformers.__version__)

4.13.0.dev0


In [ ]:
model_checkpoint = "t5-small"

In [ ]:
from datasets import load_dataset, load_metric

metric = load_metric("rouge")

In [ ]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each predictions
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_agregator: Return aggregates if this is set to True
Retu

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""


In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples_text):
    inputs = [prefix + str(doc) for doc in examples_text]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    return model_inputs
def preprocess_function_label(examples_labels):
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples_labels, max_length=max_target_length, truncation=True)
    return labels
    

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import ast
from matplotlib.pyplot import figure
from collections import defaultdict
from tqdm.auto import tqdm
import random
import torch

In [ ]:
!pip install gdown

In [ ]:
!gdown https://drive.google.com/uc?id=1mJT4TPSWcJZRvzDpNcBdnPy-NQmE9lNk

Downloading...
From: https://drive.google.com/uc?id=1mJT4TPSWcJZRvzDpNcBdnPy-NQmE9lNk
To: /content/augmented_recipes.csv
100% 501M/501M [00:01<00:00, 292MB/s]


In [ ]:
import pandas as pd
model_data_df = pd.read_csv('augmented_recipes.csv', index_col=1)

In [ ]:
train_texts=model_data_df['steps']
train_labels_list=model_data_df['tags']

In [ ]:
model_data_df['steps'][1]

'Preheat oven to 425 degrees F.\nPress dough into the bottom and sides of a 12 inch pizza pan.\nBake for 5 minutes until set but not browned.\nCut sausage into small pieces.\nWhisk eggs and milk in a bowl until frothy.\nSpoon sausage over baked crust and sprinkle with cheese.\nPour egg mixture slowly over sausage and cheese.\nS& P to taste.\nBake 15-20 minutes or until eggs are set and crust is brown.'

In [ ]:
train_labels=[]
for i in train_labels_list:
    tags_summary=i.replace(',','').replace('\'','').replace('[','').replace(']','')
    train_labels.append(tags_summary)

In [ ]:
from sklearn.model_selection import train_test_split
train_steps, test_steps, train_tags, test_tags = train_test_split(train_texts, train_labels, test_size=0.20)

In [ ]:
train_encodings=preprocess_function(train_steps)

In [ ]:
train_tags_encodings=preprocess_function_label(train_tags)
train_label_encodings=train_tags_encodings['input_ids']

In [ ]:
test_encodings=preprocess_function(test_steps)
test_tags_encodings=preprocess_function_label(test_tags)
test_label_encodings=test_tags_encodings['input_ids']

In [ ]:

class FoodComDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = FoodComDataset(train_encodings, train_label_encodings)
val_dataset = FoodComDataset(test_encodings, test_label_encodings)


In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 10
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum-proplus",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=20,
    predict_with_generate=True,
    push_to_hub=True,
    fp16=True,
    eval_accumulation_steps=1,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/t5-small-finetuned-xsum-proplus is already a clone of https://huggingface.co/Teepika/t5-small-finetuned-xsum-proplus. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp fp16 backend


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('corpus')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading corpus: Package 'corpus' not found in index


False

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 178748
  Num Epochs = 20
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 357500
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1357: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.112600,0.954671,38.804800,22.345000,33.817800,33.812000,18.333800
2,0.945700,0.818165,40.455600,23.730400,35.520400,35.516400,18.273100
3,0.864100,0.754082,41.806300,24.592600,36.460600,36.451800,18.476600
4,0.813200,0.716817,41.965900,24.741900,36.829600,36.827400,18.414600
5,0.785400,0.691219,42.227300,24.909200,37.112000,37.107300,18.461100
6,0.766100,0.672171,42.813300,25.417300,37.697900,37.691700,18.380600
7,0.739700,0.659200,44.070700,26.657300,38.743000,38.739000,18.235500
8,0.723000,0.647439,43.527400,26.367700,38.294800,38.289300,18.299000
9,0.714100,0.638523,43.870000,26.500000,38.746600,38.743000,18.310600
10,0.704800,0.631282,44.403200,26.967700,39.172800,39.170800,18.261500


Saving model checkpoint to t5-small-finetuned-xsum-proplus/checkpoint-500
Configuration saved in t5-small-finetuned-xsum-proplus/checkpoint-500/config.json
Model weights saved in t5-small-finetuned-xsum-proplus/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-xsum-proplus/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-xsum-proplus/checkpoint-500/special_tokens_map.json
tokenizer config file saved in t5-small-finetuned-xsum-proplus/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-xsum-proplus/special_tokens_map.json
Several commits (3) will be pushed upstream.
Deleting older checkpoint [t5-small-finetuned-xsum-proplus/checkpoint-17500] due to args.save_total_limit
Saving model checkpoint to t5-small-finetuned-xsum-proplus/checkpoint-1000
Configuration saved in t5-small-finetuned-xsum-proplus/checkpoint-1000/config.json
Model weights saved in t5-small-finetuned-xsum-proplus/checkpoint-1000/pytorc

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.112600,0.954671,38.804800,22.345000,33.817800,33.812000,18.333800
2,0.945700,0.818165,40.455600,23.730400,35.520400,35.516400,18.273100
3,0.864100,0.754082,41.806300,24.592600,36.460600,36.451800,18.476600
4,0.813200,0.716817,41.965900,24.741900,36.829600,36.827400,18.414600
5,0.785400,0.691219,42.227300,24.909200,37.112000,37.107300,18.461100
6,0.766100,0.672171,42.813300,25.417300,37.697900,37.691700,18.380600
7,0.739700,0.659200,44.070700,26.657300,38.743000,38.739000,18.235500
8,0.723000,0.647439,43.527400,26.367700,38.294800,38.289300,18.299000
9,0.714100,0.638523,43.870000,26.500000,38.746600,38.743000,18.310600
10,0.704800,0.631282,44.403200,26.967700,39.172800,39.170800,18.261500




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=357500, training_loss=0.7705388072620739, metrics={'train_runtime': 61262.292, 'train_samples_per_second': 58.355, 'train_steps_per_second': 5.836, 'total_flos': 3.664206576601006e+17, 'train_loss': 0.7705388072620739, 'epoch': 20.0})

In [ ]:
pwd

'/content'

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
! cp -R /content/t5-small-finetuned-xsum-proplus /content/drive/MyDrive/t5-small-finetuned-xsum-proplus

In [ ]:
!ls -ltr

total 489580
drwxr-xr-x 1 root root      4096 Oct 26 13:34 sample_data
drwxr-xr-x 4 root root      4096 Nov  2 00:50 t5-small-finetuned-xsum-gcloud1
-rw-r--r-- 1 root root 501306129 Nov  2 02:00 augmented_recipes.csv
drwxr-xr-x 5 root root      4096 Nov  2 18:42 t5-small-finetuned-xsum-proplus
drwx------ 6 root root      4096 Nov  2 20:22 drive


In [ ]:
!pwd

/content


In [ ]:
!cd /content/t5-small-finetuned-xsum-proplus/checkpoint-357500/

In [ ]:
!cd /content/t5-small-finetuned-xsum-proplus/checkpoint-357500/

In [ ]:
!git status

fatal: not a git repository (or any of the parent directories): .git


In [ ]:
import os

In [ ]:
DIRECTORY = '/content/t5-small-finetuned-xsum-proplus/' 

if os.getcwd() != DIRECTORY:
  os.chdir(DIRECTORY)

In [ ]:
!git status

On branch main
Your branch is ahead of 'origin/main' by 3 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   ../runs/Nov02_02-01-23_881d6d578c6e/events.out.tfevents.1635818494.881d6d578c6e.1767.0

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git log

commit eda00e7ee51613089f743a179580fc26bb127ace (HEAD -> main)
Author: R-M <teepika.ramasamymarimuthu@sjsu.edu>
Date:   Tue Nov 2 02:02:31 2021 +0000

    Training in progress, step 500

commit 4d6dccc4053bca2e405030f228fcc61e96f786e3
Author: R-M <teepika.ramasamymarimuthu@sjsu.edu>
Date:   Tue Nov 2 01:02:26 2021 +0000

    Training in progress, step 500

commit e7d059cbc704a7b7d6ed18e1365f2a9bc1bdc274
Author: R-M <teepika.ramasamymarimuthu@sjsu.edu>
Date:   Tue Nov 2 00:54:00 2021 +0000

    Training in progress, step 500

commit 698dca2d9f5fd77a474bde944a12da68e3a391fc (origin/main, origin/HEAD)
Author: system <system@huggingface.co>
Date:   Tue Nov 2 00:52:07 2021 +0000

    initial commit


In [ ]:
!git push https://user:$(cat /root/.huggingface/token)@huggingface.co/Teepika/t5-small-finetuned-xsum-proplus

Git LFS: (0 of 0 files, 18 skipped) 0 B / 0 B, 692.67 MB skippedCounting objects: 44, done.
Delta compression using up to 8 threads.
Compressing objects: 100% (43/43), done.
Writing objects: 100% (44/44), 558.49 KiB | 5.76 MiB/s, done.
Total 44 (delta 6), reused 0 (delta 0)
To https://huggingface.co/Teepika/t5-small-finetuned-xsum-proplus
   698dca2..eda00e7  main -> main


In [ ]:
!shasum -a 512 config.json

a6566bb93b8983776be0e960552aeaaf2d13a482a7bf1f8ad7d7b257b0ec82f469d9bc732ee91ad33fe4fe0ab8e053c85b5dbece9e1b04f9c741b072fac83945  config.json


In [ ]:
!shasum -a 512 ../config.json

a6566bb93b8983776be0e960552aeaaf2d13a482a7bf1f8ad7d7b257b0ec82f469d9bc732ee91ad33fe4fe0ab8e053c85b5dbece9e1b04f9c741b072fac83945  ../config.json


In [ ]:
!pwd

/content/t5-small-finetuned-xsum-proplus


In [ ]:
!git status

On branch main
Your branch is ahead of 'origin/main' by 3 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   runs/Nov02_02-01-23_881d6d578c6e/events.out.tfevents.1635818494.881d6d578c6e.1767.0

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!ls -ltr

total 237792
drwxr-xr-x 7 root root      4096 Nov  2 02:01 runs
-rw-r--r-- 1 root root      1385 Nov  2 02:02 config.json
-rw-r--r-- 1 root root      1924 Nov  2 02:02 tokenizer_config.json
-rw-r--r-- 1 root root 242085627 Nov  2 02:02 pytorch_model.bin
-rw-r--r-- 1 root root      1786 Nov  2 02:02 special_tokens_map.json
-rw-r--r-- 1 root root   1387489 Nov  2 02:02 tokenizer.json
-rw-r--r-- 1 root root      2991 Nov  2 02:02 training_args.bin
drwxr-xr-x 2 root root      4096 Nov  2 18:42 checkpoint-357500


In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization", model="Teepika/t5-small-finetuned-xsum-proplus")


https://huggingface.co/Teepika/t5-small-finetuned-xsum-proplus/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp1oee4k4y


Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

storing https://huggingface.co/Teepika/t5-small-finetuned-xsum-proplus/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/e2c4ca5f8cb2a7952d22542861920179cddfc82df7ff4504cdc74de183a16a34.6d4448604da4186d665c8b8853330ba7e01cdde8f8cfa1b9d269e2380b0d7694
creating metadata file for /root/.cache/huggingface/transformers/e2c4ca5f8cb2a7952d22542861920179cddfc82df7ff4504cdc74de183a16a34.6d4448604da4186d665c8b8853330ba7e01cdde8f8cfa1b9d269e2380b0d7694
loading configuration file https://huggingface.co/Teepika/t5-small-finetuned-xsum-proplus/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e2c4ca5f8cb2a7952d22542861920179cddfc82df7ff4504cdc74de183a16a34.6d4448604da4186d665c8b8853330ba7e01cdde8f8cfa1b9d269e2380b0d7694
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id"

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

storing https://huggingface.co/Teepika/t5-small-finetuned-xsum-proplus/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/612fcdcb7ac8385ce4930c895bfee58ca2aad51273634474240d79968a7c34e3.07ca42e2f4857faf7a39e7e4c451be127ba15e9242030381c5ae1602a8736af4
creating metadata file for /root/.cache/huggingface/transformers/612fcdcb7ac8385ce4930c895bfee58ca2aad51273634474240d79968a7c34e3.07ca42e2f4857faf7a39e7e4c451be127ba15e9242030381c5ae1602a8736af4
loading weights file https://huggingface.co/Teepika/t5-small-finetuned-xsum-proplus/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/612fcdcb7ac8385ce4930c895bfee58ca2aad51273634474240d79968a7c34e3.07ca42e2f4857faf7a39e7e4c451be127ba15e9242030381c5ae1602a8736af4
All model checkpoint weights were used when initializing T5ForConditionalGeneration.

All the weights of T5ForConditionalGeneration were initialized from the model checkpoint at Teepika/t5-small-finetuned-xsum-proplus.
If your

...

storing https://huggingface.co/Teepika/t5-small-finetuned-xsum-proplus/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/b5631308a32db6b3c8d4a468aecb97521a2c297da7f63b379a25db5ae19ffa61.b36b700ce91a4b99604e51a8f1694160d98bbbe9d04aee9a964bd9c86d8e4c2e
creating metadata file for /root/.cache/huggingface/transformers/b5631308a32db6b3c8d4a468aecb97521a2c297da7f63b379a25db5ae19ffa61.b36b700ce91a4b99604e51a8f1694160d98bbbe9d04aee9a964bd9c86d8e4c2e
https://huggingface.co/Teepika/t5-small-finetuned-xsum-proplus/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp2zsa8jqh


Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

storing https://huggingface.co/Teepika/t5-small-finetuned-xsum-proplus/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/aaaefe5363f4a8e16403ef0c36152929b424979c72f0cd994b004596d4e8f21b.d5d9962ef900835aadd4da96cb73224a1935218df57fc612b0585098280be3b4
creating metadata file for /root/.cache/huggingface/transformers/aaaefe5363f4a8e16403ef0c36152929b424979c72f0cd994b004596d4e8f21b.d5d9962ef900835aadd4da96cb73224a1935218df57fc612b0585098280be3b4
https://huggingface.co/Teepika/t5-small-finetuned-xsum-proplus/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp5j1vyf1c


Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

storing https://huggingface.co/Teepika/t5-small-finetuned-xsum-proplus/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/6c453b3069e61dc8500711688273f7f6510be92a0e251fabfe52b37bb1890612.c94798918c92ded6aeef2d2f0e666d2cc4145eca1aa6e1336fde07f2e13e2f46
creating metadata file for /root/.cache/huggingface/transformers/6c453b3069e61dc8500711688273f7f6510be92a0e251fabfe52b37bb1890612.c94798918c92ded6aeef2d2f0e666d2cc4145eca1aa6e1336fde07f2e13e2f46
loading file https://huggingface.co/Teepika/t5-small-finetuned-xsum-proplus/resolve/main/spiece.model from cache at None
loading file https://huggingface.co/Teepika/t5-small-finetuned-xsum-proplus/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/aaaefe5363f4a8e16403ef0c36152929b424979c72f0cd994b004596d4e8f21b.d5d9962ef900835aadd4da96cb73224a1935218df57fc612b0585098280be3b4
loading file https://huggingface.co/Teepika/t5-small-finetuned-xsum-proplus/resolve/main/added_tokens.json fro

In [ ]:
summarizer(["*NOTE: Two separate seasoning suggestions are given. Make a choice and proceed with recipe.Depending on size of squash, cut into half or fourths. Remove seeds.For spicy squash, drizzle (roughly 1/2 teaspoon) olive oil or melted butter over each cut squash piece. Season with Mexican Seasoning Mix II. For sweet squash, drizzle (roughly 1/2 teaspoon) melted honey, butter, grated piloncillo (or any combination of) over each cut squash piece. Season with Sweet Mexican Spice Mix. Bake at 350 degrees, again depending on size, for 40 minutes up to an hour, until a fork can easily pierce the skin. Be careful not to burn the squash especially if you opt to use sugar or butter. If you feel more comfortable, cover the squash with aluminum foil the first half hour, give or take, of baking.If desired, season with salt. Note for Vegan use piloncillo.","Preheat oven to 425 degrees F.Press dough into the bottom and sides of a 12 inch pizza pan. Bake for 5 minutes until set but not browned. Cut sausage into small pieces. Whisk eggs and milk in a bowl until frothy. Spoon sausage over baked crust and sprinkle with cheese. Pour egg mixture slowly over sausage and cheese. S& P to taste. Bake 15-20 minutes or until eggs are set and crust is brown."])

Disabling tokenizer parallelism, we're using DataLoader multithreading already
Your max_length is set to 200, but you input_length is only 97. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


[{'summary_text': 'appetizers squash low-cholesterol low-saturated-fat low-protein low-carb low-calorie low-fat high-protein high-fat easy'},
 {'summary_text': 'main-dish 30-minutes-or-less low-cholesterol low-saturated-fat low-protein low-carb low-fat easy'}]